In [29]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import requests
import json
import ipyvuetify as viewtify

my_headers = {'accept' : 'application/json'}
debug_flag = False

def debug(*args):
    if (debug_flag == True):
        for arg in args:
            print(arg)

# Initialize the app title, help Text widget and start_value Text widget

file           = open("images/qudt_logo-300x110.png", "rb")
image          = file.read()
label_style    = {'description_width' : 'initial'}
title_text     = viewtify.Html(tag = "div", children = ['Unit Conversion Tool'], style_ = " color: #194188; font-family: helvetica; font-size: 20px")
titleBox       = widgets.HBox([widgets.Image(value = image, format = 'png', width = 75, height = 27), title_text])
help_label     = widgets.HTML(value = '<a href = "help/help.html"><b>Help</b></a>')
start_value    = widgets.Text(value = '1', description = 'Value to convert', style = label_style, disabled = False)
qk_text        = widgets.Text(value = '', description = 'Type quantity kind name', style = label_style, disabled = False)

# qk_get_request: Constructs and issues the quantity kind SPARQL query, converts to json, 
#                 and parses the ressulting results and bindings values. The bindings are returned.
#
# RETURN:    qk_binds - the SPARQL query bindings array
# CALLED BY: update_qk_options_from_input_text

def qk_get_request():
    qk_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/schema/qudt/quantitykind/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> PREFIX fn: <http://www.w3.org/2005/xpath-functions%23> SELECT ?qk ?qkl ?qks WHERE { ?qk a qudt:QuantityKind . ?qk rdfs:label ?qkl . ?qk qudt:symbol ?qks . FILTER (fn:starts-with(?qkl, %22"
    qk_query_end   = "%22)) . } ORDER BY ?qkl"
    qk_query       = qk_query_start + qk_text.value + qk_query_end
    qk_response    = requests.get(qk_query, headers = my_headers)
    qk_data        = qk_response.json()
    qk_results     = qk_data['results']
    qk_binds       = qk_results['bindings']

    return qk_binds

# get_qk_values: Parses the 'qk' value array from the SPARQL query results.
#
# ARGUMENTS: obj     - a bindings array
# RETURN:    qlabels - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_qk_values(obj):
    qk_values = []
    i       = 0

    for item in obj:
        elem = item['qk']
        qk_values.append(elem['value'])
        i += 1

    return qk_values

# get_qk_labels: Parses the 'qkl' label array from the SPARQL query results.
#
# ARGUMENTS: obj     - a bindings array
# RETURN:    qlabels - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_qk_labels(obj):
    qk_labels = []
    i         = 0

    for item in obj:
        elem = item['qkl']
        qk_labels.append(elem['value'])
        i += 1

    return qk_labels

# get_qk_symbols: Parses the 'qk' symbol array from the SPARQL query results.
#
# ARGUMENTS: obj       - a bindings array
# RETURN:    qksymbols - the quantity kind SPARQL query symbols array
# CALLED BY: statement below

def get_qk_symbols(obj):
    qk_symbols = []
    i       = 0

    for item in obj:
        elem = item['qks']
        qk_symbols.append(elem['value'])
        i += 1

    return qk_symbols

# get_qk_labels_and_symbols: Concatenates quantity kind labels and symbols.
#
# ARGUMENTS: qkls, qkss - the labels and symbols arrays
# RETURN:    qklsss     - the concatenation of labels and symbols array
# CALLED BY: statement below

def get_qk_labels_and_symbols(labels, symbols):
    qklsss = []
    i      = 0

    while i < len(labels):
        lelem = labels[i]
        selem = symbols[i]
        qklsss.append(lelem + " - " + selem)
        i += 1

    return qklsss

# Initialize the start_unit_dd content (options) by calling the request and constructing the values and labels arrays

qk_binds       = qk_get_request()
qk_values      = get_qk_values(qk_binds)
qk_labels      = get_qk_labels(qk_binds)
qk_symbols     = get_qk_symbols(qk_binds)
qk_values_cur  = qk_values
qk_labels_cur  = qk_labels
qk_symbols_cur = qk_symbols
qk_ls_and_ss   = get_qk_labels_and_symbols(qk_labels, qk_symbols)
qk_text_len    = 0

# Construct the ls_qk_dd Dropdown widget

qk_dd = widgets.Dropdown(
    options     = qk_labels, #qk_ls_and_ss,
    description = 'Possible quantity kinds:',
    style       = label_style,
    disabled    = False,
)

# Construct the qkv_dd Dropdown widget for values. This is not used. It is a place to store values and is a HACK.

qkv_dd = widgets.Dropdown(
    options     = qk_values,
    description = 'Possible quantity kinds values:',
    style       = label_style,
    disabled    = True,
)

# Construct the qks_dd Dropdown widget for values. This is not used. It is a place to store values and is a HACK.

qks_dd = widgets.Dropdown(
    options     = qk_symbols,
    description = 'Possible quantity kinds symbols:',
    style       = label_style,
    disabled    = True,
)

# restrict_qk_labels: Restricts the content of the ls_qk_dd dropdown options in ls_qk_labels
#                     list and the qk_text.
#
# CALLED BY: update_qk_options_from_input_text
# CALLS:     [python] startswith, append

def restrict_qk_labels(vlist, llist, slist, txt):
    debug('In restrict_qk_labels - Start', llist[0], vlist[0])
    global qk_values_cur
    global qk_labels_cur
    global qk_symbols_cur
    filtered_values  = []
    filtered_labels  = []
    filtered_symbols = []
    i                = 0
    qk_values_cur    = vlist
    qk_labels_cur    = llist
    qk_symbols_cur   = slist
    while i < len(llist):
        item = llist[i]
        if item.startswith(txt):
            filtered_values.append(vlist[i])
            filtered_labels.append(llist[i])
            filtered_symbols.append(slist[i])
        i += 1

    debug('In restrict_qk_labels - End', filtered_labels[0], filtered_values[0])
    return filtered_labels, filtered_symbols, filtered_values

# update_options_from_input_text: Performs qk model update downstream of a changed selection in the qk_text
#                                 text box.
#
# ALGORITHM: (1) Declare quk_text_len as global so that we can modify it here
#            (2) Test to see if the length of qk_text.value is larger than qk_text_len (i.e., did we add chars)
#                (a) If so, define qk_values_start, qk_labels_start, and qk_symbols_start based on the current
#                    options lists
#                (b) Else if the length of qk_text.value is positive define them based on the last current lists
#                (c) Otherwise define them based on the original list (means we back spaced to empty text)
#            (3) Call restrict_qk_labels on the '_start' lists
#            (4) Test the length of result[0] to see if it is positive
#                (a) If so, define the dropdown lists values based on the results
#                (b) Otherwise reset the value of qk_text.value to be the old value minus the last character
#            (5) Call update_options_from_selected_quantity
#
# CALLED BY: on_change_qkt
# CALLS:     restrict_qk_labels

def update_qk_options_from_input_text(*args): # *args represent zero (case here) or more arguments.
    global qk_text_len
    qk_text_v = qk_text.value if qk_text.value else ""
    if len(qk_text_v) > qk_text_len:
        qk_values_start  = qkv_dd.options # qk_values
        qk_labels_start  = qk_dd.options  # qk_labels # qk_dd.options
        qk_symbols_start = qks_dd.options # qk_symbols
    elif len(qk_text_v) > 0:
        qk_values_start  = qk_values_cur
        qk_labels_start  = qk_labels_cur
        qk_symbols_start = qk_symbols_cur
    else:
        qk_values_start  = qk_values
        qk_labels_start  = qk_labels
        qk_symbols_start = qk_symbols
    qk_text_len = len(qk_text.value)
    result      = restrict_qk_labels(qk_values_start, qk_labels_start, qk_symbols_start, qk_text_v)
    if len(result[0]) > 0:
        qk_dd.options  = result[0]
        qks_dd.options = result[1]
        qkv_dd.options = result[2]
    else:
        qk_text_start = qk_text.value
        qk_text.value = qk_text_start[0: qk_text_len - 1:1]
    # update everything else
    update_options_from_selected_quantity()

# on_change_qkt: Handles changes to the quantity kind input text widget
#
# CALLED BY: Changes to qk_text value
# CALLS:     update_qk_options_from_input_text

def on_change_qkt(change):
    selected_qk_text  = qk_text.value # Not needed or used

    if change['type'] == 'change' and change['name'] == 'value':
        update_qk_options_from_input_text()

qk_text.observe(on_change_qkt)

# start_units_get_request: Constructs and issues the start unit SPARQL query, converts to json, and parses
#                          the ressulting results and bindings values. The bindings are returned.
#
# ARGUMENTS: selected_quant - the currently selected quantity
# RETURN:    stubinds      - the SPARQL query bindings array
# CALLED BY: update_options_from_selected_quantity

def start_units_get_request(selected_qk):
    stunit_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/schema/qudt/quantitykind/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?stunit ?stunit_label WHERE { <"
    stunit_query_end   = "> qudt:applicableUnit ?stunit . ?stunit rdfs:label ?stunit_label . }"
    stunit_query       = stunit_query_start + selected_qk + stunit_query_end
    stunit_response    = requests.get(stunit_query, headers = my_headers)
    studata            = stunit_response.json()
    sturesults         = studata['results']
    stubinds           = sturesults['bindings']

    return stubinds

# get_stuvalues: Parses the 'stunit' value array from the SPARQL query results.
#
# ARGUMENTS: obj       - a bindings array
# RETURN:    stuvalues - the starting unit SPARQL query values array
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit

def get_stuvalues(obj):
    stuvalues = []
    i         = 0

    for item in obj:
        elem = item['stunit']
        stuvalues.append(elem['value'])
        i += 1

    return stuvalues

# get_stulabels: Parses the 'stunit_label' value array from the SPARQL query results.
#
# ARGUMENTS: obj       - a bindings array
# RETURN:    stulabels - the start unit SPARQL query labels array
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit

def get_stulabels(obj):
    stulabels = []
    i         = 0

    for item in obj:
        elem = item['stunit_label']
        stulabels.append(elem['value'])
        i += 1

    return stulabels

# Initialize the start_unit_dd content (options) by calling the request and constructing the values and labels arrays

stubinds      = start_units_get_request(qk_values[qk_dd.index])
stunit_values = get_stuvalues(stubinds)
stunit_labels = get_stulabels(stubinds)

# Construct the start_unit_dd Dropdown widget

start_unit_dd = widgets.Dropdown(
    options     = stunit_labels,
    description = 'Starting Unit:',
    style       = label_style,
    disabled    = False,
)

# target_units_get_request: Constructs and issues the target unit SPARQL query, converts to json, and parses
#                           the ressulting results and bindings values. The bindings are returned.
#
# ARGUMENTS: selected_quant - the currently selected quantity
#            stunits        - the start_unit_dd values array
# RETURN:    tgtubinds      - the SPARQL query bindings array
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit

def target_units_get_request(selected_qk, stunits):
    tgtunit_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/schema/qudt/quantitykind/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?tgtunit ?tgtunit_label WHERE { <"
    tgtunit_query_end   = "> qudt:applicableUnit ?tgtunit . ?tgtunit rdfs:label ?tgtunit_label . }"
    tgtunit_query       = tgtunit_query_start + selected_qk + tgtunit_query_end
#    tgtunit_query_start  = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/> PREFIX unit: <http://qudt.org/vocab/unit/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?tgtunit ?tgtunit_label WHERE { <"
#    tgtunit_query_middle = "> qudt:applicableUnit ?tgtunit . ?tgtunit rdfs:label ?tgtunit_label . FILTER (?tgtunit != <"
#    stunit_value         = stunits[start_unit_dd.index]
#    tgtunit_query_end    = ">) . }"
#    tgtunit_query        = tgtunit_query_start + selected_qk + tgtunit_query_middle + stunit_value + tgtunit_query_end
    tgtunit_response     = requests.get(tgtunit_query, headers = my_headers)
    tgtudata             = tgtunit_response.json()
    tgturesults          = tgtudata['results']
    tgtubinds            = tgturesults['bindings']

    return tgtubinds

# get_tgtuvalues: Parses the 'tgtunit' value array from the SPARQL query results.
#
# ARGUMENTS: obj       - a bindings array
# RETURN:    stuvalues - the starting unit SPARQL query values array
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit

def get_tgtuvalues(obj):
    tgtuvalues = []
    i          = 0

    for item in obj:
        elem = item['tgtunit']
        tgtuvalues.append(elem['value'])
        i += 1

    return tgtuvalues

# get_tgtulabels: Parses the 'tgtunit_label' value array from the SPARQL query results.
#
# ARGUMENTS: obj        - a bindings array
# RETURN:    tgtulabels - the target unit SPARQL query labels array
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit

def get_tgtulabels(obj):
    tgtulabels = []
    i          = 0

    for item in obj:
        elem = item['tgtunit_label']
        tgtulabels.append(elem['value'])
        i += 1

    return tgtulabels

# Initialize the quant_dd content (options) by calling the request and constructing the values and labels arrays

tgtubinds      = target_units_get_request(qk_values[qk_dd.index], stunit_values)
tgtunit_values = get_tgtuvalues(tgtubinds)
tgtunit_labels = get_tgtulabels(tgtubinds)

# Construct the quant_dd Dropdown widget

target_unit_dd = widgets.Dropdown(
    options     = tgtunit_labels,
    description = 'Target Unit:',
    style       = label_style,
    disabled    = False,
)

# starting_unit_cm_get_request: Constructs and calls the starting unit conversion multiplier SPARQL query and parses
#                               the results (and returns them). If there are no 'bindings' (empty response), then
#                               set the conversion multiplier to 1.0.
#
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit
#            update_calculations_from_selected_start_value
# CALLS:     get request method
#            json parse

def starting_unit_cm_get_request(selected_stu):
    stunitcm_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX unit: <http://qudt.org/vocab/unit/> SELECT DISTINCT ?stunitcm WHERE { <"
    stunitcm_query_end   = "> qudt:conversionMultiplier ?stunitcm . }"
    stunitcm_query       = stunitcm_query_start + selected_stu + stunitcm_query_end
    stunitcm_response    = requests.get(stunitcm_query, headers=my_headers)
    stucmdata            = stunitcm_response.json()
    stucmresults         = stucmdata['results']
    stucmbinds           = stucmresults['bindings']
    if len(stucmbinds) > 0:
        stucmelem         = stucmbinds[0]
        stucm             = stucmelem['stunitcm']
        stucmvalue        = stucm['value']
        start_unit_cm     = stucmvalue
    else:
        start_unit_cm = 1.0

    return start_unit_cm

# target_unit_cm_get_request: Constructs and calls the target unit conversion multiplier SPARQL query and parses
#                             the results (and returns them). If there are no 'bindings' (empty response), then
#                             set the conversion multiplier to 1.0.
#
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit
#            update_calculations_from_selected_start_value
# CALLS:     get request method
#            json parse

def target_unit_cm_get_request(selected_tgtu):
    tgtunitcm_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX unit: <http://qudt.org/vocab/unit/> SELECT DISTINCT ?tgtunitcm WHERE { <"
    tgtunitcm_query_end   = "> qudt:conversionMultiplier ?tgtunitcm . }"
    tgtunitcm_query       = tgtunitcm_query_start + selected_tgtu + tgtunitcm_query_end
    tgtunitcm_response    = requests.get(tgtunitcm_query, headers=my_headers)
    tgtucmdata            = tgtunitcm_response.json()
    tgtucmresults         = tgtucmdata['results']
    tgtucmbinds           = tgtucmresults['bindings']
    if len(tgtucmbinds) > 0:
        tgtucmelem         = tgtucmbinds[0]
        tgtucm             = tgtucmelem['tgtunitcm']
        tgtucmvalue        = tgtucm['value']
        target_unit_cm     = tgtucmvalue
    else:
        target_unit_cm = 1.0

    return target_unit_cm

# Initializer start_unit_cm, target_unit_cm, converted_value, and target_value Text widget

start_unit_cm   = 1
target_unit_cm  = 1
converted_value = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
target_value    = widgets.Text(value = str(converted_value), description = 'Converted value', style = label_style, disabled = True)

# update_options_from_selected_quantity: Performs model updates downstream of a changed selection in the quant_dd
#                                        dropdown. That is, it updates the start_unit_dd and target_unit_dd 
#                                        model (options) contents, the conversion multipliers, and the converted 
#                                        value.
#
# CALLED BY: on_change_qk
# CALLS:     start_units_get_request, get_stuvalues, get_stulabels
#            target_units_get_request, get_tgtuvalues, get_tgtulabels
#            starting_unit_cm_get_request
#            target_unit_cm_get_request

def update_options_from_selected_quantity(*args): # *args represent zero (case here) or more arguments.
    debug('In update_options_from_selected_quantity - Start', qkv_dd.options[0], qk_dd.options[0])
    qk_options             = qkv_dd.options
    selected_qk            = qk_options[qk_dd.index if qk_dd.index else 0] # qk_values
    debug('Selected qk', selected_qk)
    stubinds               = start_units_get_request(selected_qk)
    stunit_values          = get_stuvalues(stubinds)
    stunit_labels          = get_stulabels(stubinds)
    start_unit_dd.options  = stunit_labels
    tgtubinds              = target_units_get_request(selected_qk, stunit_values)
    tgtunit_values         = get_tgtuvalues(tgtubinds)
    tgtunit_labels         = get_tgtulabels(tgtubinds)
    target_unit_dd.options = tgtunit_labels
    selected_stu           = stunit_values[start_unit_dd.index if start_unit_dd.index else 0]
    start_unit_cm          = starting_unit_cm_get_request(selected_stu)
    selected_tgtu          = tgtunit_values[target_unit_dd.index if target_unit_dd.index else 0]
    target_unit_cm         = target_unit_cm_get_request(selected_tgtu)
    converted_value        = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value     = str(converted_value)

# There are 3 on_change variants. If the selected quantity kind changes, then everything downstream must change.
# That is on_change_q. It is triggered by changes to the selected 'index' of the quant_dd dropdown so it is observing
# that widget and, when triggered, it calls the update_options_from_selected_quantity which performs all of the required
# model updates.
#
# CALLED BY: Changes to quant_dd index
# CALLS: update_options_from_selected_quantity

def on_change_qk(change):
#    selected_qk  = qk_values[qk_dd.index if qk_dd.index else 0]

    if change['type'] == 'change' and change['name'] == 'index':
        update_options_from_selected_quantity()

qk_dd.observe(on_change_qk)

# update_options_from_selected_start_unit: Performs model updates downstream of a changed selection in the start_unit_dd
#                                          dropdown. That is, it updates the target_unit_dd contents, the conversion
#                                          multipliers, and the converted value.
#
# CALLED BY: on_change_stu
# CALLS:     target_units_get_request, get_tgtuvalues, get_tgtulabels
#            starting_unit_cm_get_request
#            target_unit_cm_get_request

def update_options_from_selected_start_unit(*args): # *args represent zero (case here) or more arguments.
    qk_options             = qkv_dd.options
    selected_qk            = qk_options[qk_dd.index if qk_dd.index else 0]
    stubinds               = start_units_get_request(selected_qk)
    stunit_values          = get_stuvalues(stubinds)
    tgtubinds              = target_units_get_request(selected_qk, stunit_values)
    tgtunit_values         = get_tgtuvalues(tgtubinds)
    tgtunit_labels         = get_tgtulabels(tgtubinds)
    target_unit_dd.options = tgtunit_labels
    selected_stu           = stunit_values[start_unit_dd.index if start_unit_dd.index else 0]
    start_unit_cm          = starting_unit_cm_get_request(selected_stu)
    selected_tgtu          = tgtunit_values[target_unit_dd.index if target_unit_dd.index else 0]
    target_unit_cm         = target_unit_cm_get_request(selected_tgtu)
    converted_value        = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value     = str(converted_value)

# on_change_su: Watches changes to start_unit_dd and calls update_options_from_selected_start_unit when the selected
#               index changes. It thus observes start_unit_dd.
#
# CALLED BY: Changes to start_unit_dd index values
# CALLS      update_options_from_selected_start_unit

def on_change_stu(change):
#    selected_qk  = qk_values[qk_dd.index if qk_dd.index else 0]

    if change['type'] == 'change' and change['name'] == 'index':
        update_options_from_selected_start_unit()

start_unit_dd.observe(on_change_stu)

# update_options_from_selected_target_unit: Performs model updates downstream of a changed selection in the target_unit_dd
#                                           dropdown. That is, it updates the target_unit_dd contents, the conversion
#                                           multipliers, and the converted value.
#
# CALLED BY: on_change_tgtu
# CALLS:     target_units_get_request, get_tgtuvalues, get_tgtulabels
#            starting_unit_cm_get_request
#            target_unit_cm_get_request

def update_options_from_selected_target_unit(*args): # *args represent zero (case here) or more arguments.
    qk_options             = qkv_dd.options
    selected_qk            = qk_options[qk_dd.index if qk_dd.index else 0]
    stubinds               = start_units_get_request(selected_qk)
    stunit_values          = get_stuvalues(stubinds)
    tgtubinds              = target_units_get_request(selected_qk, stunit_values)
    tgtunit_values         = get_tgtuvalues(tgtubinds)
    tgtunit_labels         = get_tgtulabels(tgtubinds)
    target_unit_dd.options = tgtunit_labels
    selected_stu           = stunit_values[start_unit_dd.index if start_unit_dd.index else 0]
    start_unit_cm          = starting_unit_cm_get_request(selected_stu)
    selected_tgtu          = tgtunit_values[target_unit_dd.index if target_unit_dd.index else 0]
    target_unit_cm         = target_unit_cm_get_request(selected_tgtu)
    converted_value        = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value     = str(converted_value)

# on_change_tgtu: Watches changes to target_unit_dd and calls update_options_from_selected_start_unit when the selected
#                 index changes. It thus observes target_unit_dd.
#
# CALLED BY: Changes to target_unit_dd index values
# CALLS      update_options_from_selected_target_unit

def on_change_tgtu(change):
    if change['type'] == 'change' and change['name'] == 'index':
        update_options_from_selected_target_unit()

target_unit_dd.observe(on_change_tgtu)

# update_calculations_from_selected_start_value: Performs model updates downstream of a changed value in the start_unit_dd
#                                                dropdown. That is, it updates the conversion multipliers, and the converted value.
#
# CALLED BY: on_change_suv
# CALLS:     starting_unit_cm_get_request
#            target_unit_cm_get_request

def update_calculations_from_selected_start_value(*args): # *args represent zero (case here) or more arguments.
    qk_options         = qkv_dd.options
    selected_qk        = qk_options[qk_dd.index if qk_dd.index else 0]
    stubinds           = start_units_get_request(selected_qk)
    stunit_values      = get_stuvalues(stubinds)
    selected_stu       = stunit_values[start_unit_dd.index if start_unit_dd.index else 0]
    start_unit_cm      = starting_unit_cm_get_request(selected_stu) # if starting_unit_cm_get_request() != 'None' else 1
    tgtubinds          = target_units_get_request(selected_qk, stunit_values)
    tgtunit_values     = get_tgtuvalues(tgtubinds)
    selected_tgtu      = tgtunit_values[target_unit_dd.index if target_unit_dd.index else 0]
    target_unit_cm     = target_unit_cm_get_request(selected_tgtu)   # if target_unit_cm_get_request() != 'None' else 1
    converted_value    = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value = str(converted_value)


# on_changes_suv: Watches changes to start_value and updates conversion multipliers and converted values when it
#                 changes.
#
# CALLS: update_calculations_from_selected_start_value

def on_change_suv(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_calculations_from_selected_start_value()

start_value.observe(on_change_suv)

dyn_grid_items  = [qk_dd, help_label, start_unit_dd, target_unit_dd, start_value, target_value]
dyn_box         = widgets.GridBox(dyn_grid_items, layout=widgets.Layout(grid_template_columns="repeat(2,325px)"))
app_box         = widgets.VBox([titleBox, qk_text, dyn_box])
display(app_box)
